In [12]:
import io
import os
import re
import glob
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from causallearn.utils.GraphUtils import GraphUtils
import matplotlib.image as mpimg
plt.style.use('seaborn')

In [3]:
# Specify the data path and experiment name
path = "../linear/data/pcm/random_input"
exp = "exp-7"

In [27]:
# Find all the csv files under the target directory.
def list_csv_files(directory):
    """
    List all CSV files in the specified directory without traversing into subdirectories.

    Args:
        directory (str): The path to the directory where the search should be performed.

    Returns:
        list: A list of paths to CSV files found in the directory.
    """
    # Pattern to match all CSV files in the directory
    pattern = f"{directory}/*.csv"
    
    # Use glob.glob to find all files matching the pattern
    csv_files = glob.glob(pattern)
    
    return csv_files

csv_files = list_csv_files(os.path.join(path, exp))
print("CSV files found:")
for csv_file in csv_files:
    print(csv_file)

CSV files found:
../linear/data/pcm/random_input/exp-7/rx-pcm.csv
../linear/data/pcm/random_input/exp-7/bridge-pcm.csv
../linear/data/pcm/random_input/exp-7/pcm-pcie.csv
../linear/data/pcm/random_input/exp-7/ndpi_stats-pcm.csv
../linear/data/pcm/random_input/exp-7/nf_router-pcm.csv
../linear/data/pcm/random_input/exp-7/tx_stats.csv
../linear/data/pcm/random_input/exp-7/tx-pcm.csv
../linear/data/pcm/random_input/exp-7/rx_stats.csv
../linear/data/pcm/random_input/exp-7/firewall-pcm.csv
../linear/data/pcm/random_input/exp-7/latency.csv
../linear/data/pcm/random_input/exp-7/pcm-memory.csv
../linear/data/pcm/random_input/exp-7/payload_scan-pcm.csv
../linear/data/pcm/random_input/exp-7/nf_out.csv


In [38]:
vnfs = []
for f in csv_files:
    if os.path.basename(f) == "latency.csv": path_latency = f
    elif os.path.basename(f) == "rx_stats.csv": path_rx = f
    elif os.path.basename(f) == "tx_stats.csv": path_tx = f
    elif os.path.basename(f) == "nf_out.csv": path_nfout = f
    elif os.path.basename(f) == "pcm-memory.csv": path_mem = f
    elif os.path.basename(f) == "pcm-pcie.csv": path_pcie = f
    else:
        vnfs.append(f)

In [41]:
vnfs

['../linear/data/pcm/random_input/exp-7/rx-pcm.csv',
 '../linear/data/pcm/random_input/exp-7/bridge-pcm.csv',
 '../linear/data/pcm/random_input/exp-7/ndpi_stats-pcm.csv',
 '../linear/data/pcm/random_input/exp-7/nf_router-pcm.csv',
 '../linear/data/pcm/random_input/exp-7/tx-pcm.csv',
 '../linear/data/pcm/random_input/exp-7/firewall-pcm.csv',
 '../linear/data/pcm/random_input/exp-7/payload_scan-pcm.csv']

In [53]:
# Load the PCM files
for p in vnfs:
    print("VNF: ", os.path.basename(p).split("-")[0])
    
    # Read the PCM csv file for each function, using the first two rows as header fields
    df = pd.read_csv(p, header=[0,1])
    
    # Define the relevant fields that need to be extracted from the source.
    columns_filtered = [col for col in df.columns if re.match(r'Core\d',col[0])]
    print(columns_filtered)

VNF:  rx
[('Core2 (Socket 0)', 'EXEC'), ('Core2 (Socket 0)', 'IPC'), ('Core2 (Socket 0)', 'FREQ'), ('Core2 (Socket 0)', 'AFREQ'), ('Core2 (Socket 0)', 'L3MISS'), ('Core2 (Socket 0)', 'L2MISS'), ('Core2 (Socket 0)', 'L3HIT'), ('Core2 (Socket 0)', 'L2HIT'), ('Core2 (Socket 0)', 'L3MPI'), ('Core2 (Socket 0)', 'L2MPI'), ('Core2 (Socket 0)', 'L3OCC'), ('Core2 (Socket 0)', 'C0res%'), ('Core2 (Socket 0)', 'C1res%'), ('Core2 (Socket 0)', 'C3res%'), ('Core2 (Socket 0)', 'C6res%'), ('Core2 (Socket 0)', 'C7res%'), ('Core2 (Socket 0)', 'TEMP'), ('Core2 (Socket 0)', 'INST'), ('Core2 (Socket 0)', 'ACYC'), ('Core2 (Socket 0)', 'TIME(ticks)'), ('Core2 (Socket 0)', 'PhysIPC'), ('Core2 (Socket 0)', 'PhysIPC%'), ('Core2 (Socket 0)', 'INSTnom'), ('Core2 (Socket 0)', 'INSTnom%')]
VNF:  bridge
[('Core7 (Socket 0)', 'EXEC'), ('Core7 (Socket 0)', 'IPC'), ('Core7 (Socket 0)', 'FREQ'), ('Core7 (Socket 0)', 'AFREQ'), ('Core7 (Socket 0)', 'L3MISS'), ('Core7 (Socket 0)', 'L2MISS'), ('Core7 (Socket 0)', 'L3HIT'), (

In [5]:
columns_filtered = [col for col in core.columns if re.match(r'Core\d',col[0])]
columns_filtered

[('Core7 (Socket 0)', 'EXEC'),
 ('Core7 (Socket 0)', 'IPC'),
 ('Core7 (Socket 0)', 'FREQ'),
 ('Core7 (Socket 0)', 'AFREQ'),
 ('Core7 (Socket 0)', 'L3MISS'),
 ('Core7 (Socket 0)', 'L2MISS'),
 ('Core7 (Socket 0)', 'L3HIT'),
 ('Core7 (Socket 0)', 'L2HIT'),
 ('Core7 (Socket 0)', 'L3MPI'),
 ('Core7 (Socket 0)', 'L2MPI'),
 ('Core7 (Socket 0)', 'L3OCC'),
 ('Core7 (Socket 0)', 'C0res%'),
 ('Core7 (Socket 0)', 'C1res%'),
 ('Core7 (Socket 0)', 'C3res%'),
 ('Core7 (Socket 0)', 'C6res%'),
 ('Core7 (Socket 0)', 'C7res%'),
 ('Core7 (Socket 0)', 'TEMP'),
 ('Core7 (Socket 0)', 'INST'),
 ('Core7 (Socket 0)', 'ACYC'),
 ('Core7 (Socket 0)', 'TIME(ticks)'),
 ('Core7 (Socket 0)', 'PhysIPC'),
 ('Core7 (Socket 0)', 'PhysIPC%'),
 ('Core7 (Socket 0)', 'INSTnom'),
 ('Core7 (Socket 0)', 'INSTnom%')]

In [6]:
core[columns_filtered]

Core7 (Socket 0)                                                    \
                 EXEC   IPC  FREQ AFREQ L3MISS L2MISS L3HIT L2HIT L3MPI   
0                0.55  3.13  0.18  1.12    0.0   1.23   1.0  0.14   0.0   
1                0.43  2.78  0.16  1.12    0.0   1.54   1.0  0.14   0.0   
2                0.44  2.82  0.16  1.12    0.0   1.60   1.0  0.14   0.0   
3                0.42  2.75  0.15  1.12    0.0   1.56   1.0  0.14   0.0   
4                0.43  2.78  0.15  1.12    0.0   1.59   1.0  0.14   0.0   
...               ...   ...   ...   ...    ...    ...   ...   ...   ...   
1478             0.45  2.84  0.16  1.12    0.0   2.19   1.0  0.16   0.0   
1479             0.45  2.83  0.16  1.12    0.0   2.20   1.0  0.16   0.0   
1480             0.46  2.90  0.16  1.12    0.0   2.12   1.0  0.16   0.0   
1481             0.44  2.82  0.15  1.12    0.0   2.16   1.0  0.16   0.0   
1482             0.46  2.85  0.16  1.12    0.0   2.22   1.0  0.16   0.0   

              ...                                                          \
       L2MPI  ... C6res% C7res% TEMP     INST    ACYC TIME(ticks) PhysIPC   
0     0.0009  ...    0.0    0.0   49  1425.00  455.27     2599.04    6.26   
1     0.0014  ...    0.0    0.0   49  1125.87  405.15     2595.90    5.56   
2     0.0014  ...    0.0    0.0   49  1144.18  405.30     2598.07    5.65   
3     0.0014  ...    0.0    0.0   48  1101.78  401.08     2598.13    5.49   
4     0.0014  ...    0.0    0.0   47  1107.31  398.45     2595.52    5.56   
...      ...  ...    ...    ...  ...      ...     ...         ...     ...   
1478  0.0019  ...    0.0    0.0   36  1158.08  407.88     2598.10    5.68   
1479  0.0019  ...    0.0    0.0   36  1165.35  412.50     2595.53    5.65   
1480  0.0018  ...    0.0    0.0   36  1193.54  411.95     2595.47    5.79   
1481  0.0019  ...    0.0    0.0   36  1134.46  402.04     2598.54    5.64   
1482  0.0019  ...    0.0    0.0   36  1186.63  415.68     2595.78    5.71   

                                
     PhysIPC% INSTnom INSTnom%  
0      156.50    1.10    27.41  
1      138.94    0.87    21.69  
2      141.15    0.88    22.02  
3      137.35    0.85    21.20  
4      138.95    0.85    21.33  
...       ...     ...      ...  
1478   141.96    0.89    22.29  
1479   141.25    0.90    22.45  
1480   144.86    0.92    22.99  
1481   141.09    0.87    21.83  
1482   142.73    0.91    22.86  

[1483 rows x 24 columns]

In [9]:
for c in columns_filtered:
    print("bridge_" + c[1])

bridge_EXEC
bridge_IPC
bridge_FREQ
bridge_AFREQ
bridge_L3MISS
bridge_L2MISS
bridge_L3HIT
bridge_L2HIT
bridge_L3MPI
bridge_L2MPI
bridge_L3OCC
bridge_C0res%
bridge_C1res%
bridge_C3res%
bridge_C6res%
bridge_C7res%
bridge_TEMP
bridge_INST
bridge_ACYC
bridge_TIME(ticks)
bridge_PhysIPC
bridge_PhysIPC%
bridge_INSTnom
bridge_INSTnom%
